In [1]:
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [2]:
gen_data = pd.read_csv('Plant_1_Generation_Data.csv')

gen_data

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.000,6259559.0
1,15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.000,6183645.0
2,15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.000,6987759.0
3,15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.000,7602960.0
4,15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.000,7158964.0
...,...,...,...,...,...,...,...
68773,17-06-2020 23:45,4135001,uHbuxQJl8lW7ozc,0.0,0.0,5967.000,7287002.0
68774,17-06-2020 23:45,4135001,wCURE6d3bPkepu2,0.0,0.0,5147.625,7028601.0
68775,17-06-2020 23:45,4135001,z9Y9gH1T5YWrNuG,0.0,0.0,5819.000,7251204.0
68776,17-06-2020 23:45,4135001,zBIq5rxdHJRwDNY,0.0,0.0,5817.000,6583369.0


In [3]:
source_key_groups = gen_data.groupby('SOURCE_KEY')

split_datasets = {source_key: group for source_key, group in source_key_groups}

split_datasets

{'1BY6WEcLGh8j5v7':               DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  \
 0      15-05-2020 00:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
 21     15-05-2020 00:15   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
 42     15-05-2020 00:30   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
 63     15-05-2020 00:45   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
 84     15-05-2020 01:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
 ...                 ...       ...              ...       ...       ...   
 68668  17-06-2020 22:45   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
 68690  17-06-2020 23:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
 68712  17-06-2020 23:15   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
 68734  17-06-2020 23:30   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
 68756  17-06-2020 23:45   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
 
        DAILY_YIELD  TOTAL_YIELD  
 0              0.0    6259559.0  
 21      

In [4]:
min_length = min(len(group) for group in split_datasets.values())

truncated_datasets = {source_key: group.head(min_length) for source_key, group in split_datasets.items()}

truncated_datasets

{'1BY6WEcLGh8j5v7':               DATE_TIME  PLANT_ID       SOURCE_KEY     DC_POWER    AC_POWER  \
 0      15-05-2020 00:00   4135001  1BY6WEcLGh8j5v7     0.000000    0.000000   
 21     15-05-2020 00:15   4135001  1BY6WEcLGh8j5v7     0.000000    0.000000   
 42     15-05-2020 00:30   4135001  1BY6WEcLGh8j5v7     0.000000    0.000000   
 63     15-05-2020 00:45   4135001  1BY6WEcLGh8j5v7     0.000000    0.000000   
 84     15-05-2020 01:00   4135001  1BY6WEcLGh8j5v7     0.000000    0.000000   
 ...                 ...       ...              ...          ...         ...   
 67568  17-06-2020 10:15   4135001  1BY6WEcLGh8j5v7  6298.000000  617.037500   
 67590  17-06-2020 10:30   4135001  1BY6WEcLGh8j5v7  6747.000000  660.842857   
 67612  17-06-2020 10:45   4135001  1BY6WEcLGh8j5v7  6404.500000  627.412500   
 67634  17-06-2020 11:00   4135001  1BY6WEcLGh8j5v7  6260.571429  613.085714   
 67656  17-06-2020 11:15   4135001  1BY6WEcLGh8j5v7  7522.750000  736.525000   
 
        DAILY_YIELD

In [5]:
weather_data = pd.read_csv('Plant_1_Weather_Sensor_Data.csv')
weather_data

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0
...,...,...,...,...,...,...
3177,2020-06-17 22:45:00,4135001,HmiyD2TTLFNqkNe,22.150570,21.480377,0.0
3178,2020-06-17 23:00:00,4135001,HmiyD2TTLFNqkNe,22.129816,21.389024,0.0
3179,2020-06-17 23:15:00,4135001,HmiyD2TTLFNqkNe,22.008275,20.709211,0.0
3180,2020-06-17 23:30:00,4135001,HmiyD2TTLFNqkNe,21.969495,20.734963,0.0


In [6]:
weather_data['DATE_TIME'] = pd.to_datetime(weather_data['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')
for source_key, df in truncated_datasets.items():
    df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], dayfirst=True)

merged_datasets = {source_key: pd.merge(df, weather_data, on='DATE_TIME')
                   for source_key, df in truncated_datasets.items()}

merged_datasets

C:\Users\Jared\AppData\Local\Temp\ipykernel_33520\898775475.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], dayfirst=True)
C:\Users\Jared\AppData\Local\Temp\ipykernel_33520\898775475.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], dayfirst=True)
C:\Users\Jared\AppData\Local\Temp\ipykernel_33520\898775475.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


{'1BY6WEcLGh8j5v7':                DATE_TIME  PLANT_ID_x     SOURCE_KEY_x     DC_POWER  \
 0    2020-05-15 00:00:00     4135001  1BY6WEcLGh8j5v7     0.000000   
 1    2020-05-15 00:15:00     4135001  1BY6WEcLGh8j5v7     0.000000   
 2    2020-05-15 00:30:00     4135001  1BY6WEcLGh8j5v7     0.000000   
 3    2020-05-15 00:45:00     4135001  1BY6WEcLGh8j5v7     0.000000   
 4    2020-05-15 01:00:00     4135001  1BY6WEcLGh8j5v7     0.000000   
 ...                  ...         ...              ...          ...   
 3099 2020-06-17 10:15:00     4135001  1BY6WEcLGh8j5v7  6298.000000   
 3100 2020-06-17 10:30:00     4135001  1BY6WEcLGh8j5v7  6747.000000   
 3101 2020-06-17 10:45:00     4135001  1BY6WEcLGh8j5v7  6404.500000   
 3102 2020-06-17 11:00:00     4135001  1BY6WEcLGh8j5v7  6260.571429   
 3103 2020-06-17 11:15:00     4135001  1BY6WEcLGh8j5v7  7522.750000   
 
         AC_POWER  DAILY_YIELD  TOTAL_YIELD  PLANT_ID_y     SOURCE_KEY_y  \
 0       0.000000     0.000000  6259559.000     413

In [7]:
complete_dataset = pd.concat(merged_datasets.values(), ignore_index=True)

complete_json_output_path = 'merged_gen_data.json'
complete_dataset.to_json(complete_json_output_path, orient='records', date_format='iso')

complete_dataset

,DATE_TIME,PLANT_ID_x,SOURCE_KEY_x,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,PLANT_ID_y,SOURCE_KEY_y,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,1BY6WEcLGh8j5v7,0.000000,0.000000,0.000000,6259559.000,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.000000
1,2020-05-15 00:15:00,4135001,1BY6WEcLGh8j5v7,0.000000,0.000000,0.000000,6259559.000,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.000000
2,2020-05-15 00:30:00,4135001,1BY6WEcLGh8j5v7,0.000000,0.000000,0.000000,6259559.000,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.000000
3,2020-05-15 00:45:00,4135001,1BY6WEcLGh8j5v7,0.000000,0.000000,0.000000,6259559.000,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.000000
4,2020-05-15 01:00:00,4135001,1BY6WEcLGh8j5v7,0.000000,0.000000,0.000000,6259559.000,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
68279,2020-06-17 17:45:00,4135001,zVJPv84UY57bAof,958.857143,93.142857,5874.428571,7363236.429,4135001,HmiyD2TTLFNqkNe,24.174170,25.950272,0.065259
68280,2020-06-17 18:00:00,4135001,zVJPv84UY57bAof,603.000000,58.414286,5892.000000,7363254.000,4135001,HmiyD2TTLFNqkNe,24.130349,25.080925,0.041940
68281,2020-06-17 18:15:00,4135001,zVJPv84UY57bAof,340.250000,32.862500,5903.750000,7363265.750,4135001,HmiyD2TTLFNqkNe,24.038157,24.068250,0.023446
68282,2020-06-17 18:30:00,4135001,zVJPv84UY57bAof,77.714286,7.485714,5909.142857,7363271.143,4135001,HmiyD2TTLFNqkNe,23.840239,22.968658,0.007007
